In [6]:
import logging

from Loader import fileloader_proto as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
#from Loader import dfutils as utils
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

In [7]:
month = '201906'#para cargar unitarios, disminuir 1 mes
month_unitario = 201902#poner un mes mas que el de arriba.

inifile = fl.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)


INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/jsaenza/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/jsaenza/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/jsaenza/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test
INFO:    Setting database path to C:/Users/jsaenza/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


In [9]:
# Configurando los archivos de configuración para importar archivos tipo : xlsx, txt, ini
#parser = inifile.getIniFileParser()

#loader = fl.LoadFileProcess(month)
#loader.setParser(parser)
#loader.setDefaultPath(defaultpath)

# Carga de Archivos
#*******************
# Asegurate que el directorio donde está el archivo este correcto
# Asegurate que no haya pestañas ocultas en Bases GCP ni Plataformas. 
# En GCP_Base comisiones Partir en dos periodos

# Preparación : Ninguna
# File :  _RRHH Ceses
# Estado: OK
"""
section_ceses = fl.SectionObj(inifile,"Ceses",month)
loader_ceses = fl.LoadFileProcess(section_ceses)
ceses = loader_ceses.loadFile()
"""

# Preparación : Importar en periodo Mensual y quincenal
# Advertencia : Verificar que no se duplique
# File :  _Soporte Estructura Canales Venta Mercado Empresa
# Estado: OK
""""
section_jerarquia = fl.SectionObj(inifile,"Jerarquia",month)
loader_jerarquia = fl.LoadFileProcess(section_jerarquia)
jerarquia = loader_jerarquia.loadFile()
"""


# Preparación : Agregar la columna de Seguro si Gestión de Información no lo proporciona
# File :  _Planeamiento Inar Empresas
# Estado: OK
"""
section_inar = fl.SectionObj(inifile,"Inar",month)
loader_inar = fl.LoadFileProcess(section_inar)
inar = loader_inar.loadFile()
"""

# Preparación :Completar dos columnas adicionales ver archivo anterior. Mensual y Quincenal
# File : _Planeamiento Deacs Empresas
#deacs = loader.loadFile('Deacs')
"""
section_deacs = fl.SectionObj(inifile,"Deacs",month)
loader_deacs = fl.LoadFileProcess(section_deacs)
deacs = loader_deacs.loadFile()
"""


# Preparación : Completar dos columnas y Verificar Fecha. Periodo Mensual y Quincenal. Completar  vendedor_cross_selling 
# de _GCP BASE COMISIONES (BASE SVA / VENDEDOR PYME). Partir el GCE RAS en Ventas_SSAA y Deacs_SSAA
# File : _GCE Ventas SSAA
# Estado: OK
#"""
section_ventas_ssaa = fl.SectionObj(inifile,"Ventas_SSAA_new",month)
loader_ventas_ssaa = fl.LoadFileProcess(section_ventas_ssaa)
ventas_ssaa = loader_ventas_ssaa.loadFile()
ventas_ssaa = ventas_ssaa.replace(np.nan, '', regex=True)
#Eliminamos los contratos con algun valor en la columna NETO
ventas_ssaa = ventas_ssaa[ventas_ssaa['NETO'] == "" ]
#Considerar únicamente los registros que sean “Si” en la columna “Considerar”, los demás se eliminan/ no se consideran.
ventas_ssaa = ventas_ssaa[    (ventas_ssaa['CONSIDERAR'] == "Si")  ]
ventas_ssaa.reset_index(drop = True, inplace = True)
#"""


# Preparación : Completar dos columnas.Verificar Fechas. Periodo Mensual y Quincenal.
# File : _GCE Deacs SSAA
#section_deacs_ssaa = fl.SectionObj(inifile,"Deacs_SSAA_new",month)
#loader_deacs_ssaa = fl.LoadFileProcess(section_deacs_ssaa)
#deacs_ssaa = loader_deacs_ssaa.loadFile()

# Preparación : Ingresar en Periodo Mensual y Quincenal. Verificar Fechas
# File : _GCE Paquetes
# Estado: OK
"""
section_paquetes = fl.SectionObj(inifile,"Paquetes_new",month)
loader_paquetes = fl.LoadFileProcess(section_paquetes)
paquetes = loader_paquetes.loadFile()
"""

# Advertencia : la lectura del unitario es en el tiempo Mes de Comisiones -1. 
# File :  _Comisiones Pymes
"""
section_unitarios = fl.SectionObj(inifile,"Unitarios",month)
loader_unitarios = fl.LoadFileProcess(section_unitarios)
unitarios = loader_unitarios.loadFile()

unitarios = unitarios[unitarios['COMISION_UNITARIA']>0]
unitarios.reset_index(drop = True, inplace = True)
"""

#soluciones de negocio en COMISION_UNITARIA, VENDEDOR, FECHA_PROCESO


# Preparación : Completar columnas numericas Cero
# File : _Experiencia Bolsa 3G
#bolsas = loader.loadFile('Bolsas')
#Este mes no hubo

# Preparación : Ninguna
# File : _Riesgos Actividad Lima, _Riesgos Actividad Regiones
#actividad = loader.loadFile('Actividad') #en deshuso

# Preparación : Update despues de Ingresar el INAR mensual. Llenar carterización y dealer regiones
# File _ : _GCP Base Comisiones
"""
section_base_pyme = fl.SectionObj(inifile,"Bases_GCP",month)
loader_base_pyme = fl.LoadFileProcess(section_base_pyme)
basesgcp = loader_base_pyme.loadFile()
"""


# Advertencia : Separar en dos Periodos debido a Equipos de Captura Quincenal, tomar sólo el quincenal del periodo anterior
# File : _GCP Base Quincenal Comisiones
#basesgcpquincenal = loader.loadFile('Bases_GCP_Quincenal') #no se esta usando

# Preparación : Ver Errores. Cargar a conveniencia
# Advertencia : Importar en caso no lo incluya Gestión de Información
# File : _GCE Base Comisiones
#basesgce = loader.loadFile('Bases_GCE') #no se esta usando

El tamaño de Ventas_SSAA_new es 4165 registros


'\nsection_base_pyme = fl.SectionObj(inifile,"Bases_GCP",month)\nloader_base_pyme = fl.LoadFileProcess(section_base_pyme)\nbasesgcp = loader_base_pyme.loadFile()\n'

In [10]:
# Testing Dataframes

#jerarquia.dtypes

#inar.dtypes
#deacs.dtypes 

#ventas_ssaa.dtypes
#deacs_ssaa.dtypes
#paquetes.dtypes
#bolsas.dtypes

#basesgcp.dtypes
#basesgcp.describe()
#basesgcp.describe(include = ['O'])
#basesgce.dtypes
#actividad.dtypes
#basesgcpquincenal.dtypes
#jerarquia.dtypes

#unitarios.dtypes

# Looking the df
#unitarios
#unitarios[unitarios['PERIODO_ACTIVACION']==201901]
#basesgcp[basesgcp['SEGMENTO_ACCESS'].notnull()].head()
#paquetes.head()
#ceses.head()
#basesgcp
ventas_ssaa

,CODIGO,RAZON_SOCIAL,CONTRATO,SERVICE,ACTION_DATE,PHONENUMBER,ESTADO,ACCESSREAL,FAMILIA,GANADO_POR,VENDEDOR_CROSS_SELLING,EQUIPO,RUC,CUENTA,SEGMENTO,GERENCIA2,ZONA_DE_VENTA,CONSIDERAR,NETO,OBSERVACIONES
0,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,3.62029e+07,Actividades de Campo,2019-06-28,5.1959e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
1,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,3.62057e+07,Actividades de Campo,2019-06-28,5.19597e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
2,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,4.9768e+07,Actividades de Campo,2019-06-28,5.1925e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
3,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,4.9768e+07,Actividades de Campo,2019-06-28,5.1925e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
4,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,4.9768e+07,Actividades de Campo,2019-06-28,5.1925e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
5,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,4.97681e+07,Actividades de Campo,2019-06-28,5.1925e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
6,1.10496495,CONTRATISTA Y MANTENIMIENTO DE TELECOMUNICACI...,4.9768e+07,Actividades de Campo,2019-06-28,5.1925e+10,NEWS,12.710000,WAP1,GCABA2,JMARTINEZA,HARCOS,20497637431,1,Grandes Empresas AP,VENTA REGIONAL EMPRESA,REGIONAL SUR,Si,,
7,1.20609048,A T SECURITY NATIONAL S.A.C.,4.95747e+07,Alertas de Seguridad,2019-06-11,5.19774e+10,NEWS,10.170000,WAP1,GCABA2,NO DETERMINADO,HARCOS,20493433025,1,,NO DETERMINADO,NO DETERMINADO,Si,,
8,1.20609048,A T SECURITY NATIONAL S.A.C.,4.95747e+07,Alertas de Seguridad,2019-06-11,5.19774e+10,NEWS,10.170000,WAP1,GCABA2,NO DETERMINADO,HARCOS,20493433025,1,,NO DETERMINADO,NO DETERMINADO,Si,,
9,1.20609048,A T SECURITY NATIONAL S.A.C.,4.95747e+07,Alertas de Seguridad,2019-06-11,5.19774e+10,NEWS,10.170000,WAP1,GCABA2,NO DETERMINADO,HARCOS,20493433025,1,,NO DETERMINADO,NO DETERMINADO,Si,,


In [12]:
#1. Carga de Data en Base de Datos

#dbmanager.dbOperation('insert','tblJerarquia',jerarquia) #ok
#dbmanager.dbOperation('insert','tblVentas',inar)#ok
#dbmanager.dbOperation('insert','tblDeacs',deacs) #no

#dbmanager.dbOperation('insert','tblVentaSSAANew',ventas_ssaa) #ok
#dbmanager.dbOperation('insert','tblDeacSSAA',deacs_ssaa) FALTA, investigar
#dbmanager.dbOperation('insert','tblPaquetes',paquetes) #ok

#********************************************
#2.  Update de tablas de Ventas y Deacs (deben estar en memoria)

#dbmanager.dbOperation('update','Ceses', ceses)  #ok

#dbmanager.dbOperation('update','Bolsas',bolsas) #no hubo
dbmanager.dbOperation('update','SumVentaSSAA',ventas_ssaa) #
#dbmanager.dbOperation('update','Paquetes') #
#dbmanager.dbOperation('update','Unitarios', unitarios) #se debe meter en los contratos de las ventas del mes pasado

#dbmanager.dbOperation('update','Actividad', actividad) #en deshuso
#dbmanager.dbOperation('update','Bases_GCP', basesgcp)

#3. Calculando el Gross Comisión y las Reversiones (Obligatorio al final de la carga)
#*************************************************
start_time = time.time()

#dbmanager.dbOperation('update','Gross_Comision') # ok
#dbmanager.dbOperation('update','Reversiones') # ok

time_consuming = (time.time() - start_time) # In minutes
print("--- %s seconds ---" % time_consuming)

# DEPRECATED:
#dbmanager.dbOperation('update','Bases_GCP_Quincenal',basesgcpquincenal) # actualizar con data quincenal,
#dbmanager.dbOperation('update','Bases_GCE',basesgce) # Usar a conveniencia,

Los registros de la tabla SumVentaSSAA es 2905 registros actualizados 
--- 0.0 seconds ---


In [ ]:
import logging

from Loader import fileloader_proto as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
#from Loader import dfutils as utils
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

month = '201905'#para cargar unitarios, disminuir 1 mes
month_unitario = 201902#poner un mes mas que el de arriba.

inifile = fl.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)

dbmanager.dbOperation('update','Reversiones') # ok